In [76]:
from clpsych import store, data

import codecs
import os

import pandas

## Read in data from parse files (not Spacy format)

In [ ]:
tokenized = {}
i = 0

base_i = i
more = True
while more:
    if i % 10 == 0: print i
    if i < base_i + 50 and os.path.isfile('parses/{}.parse'.format(i)):
        docs, titles = read_parses('parses/{}.parse'.format(i))
        keys = [key for key in docs.keys() if key]
        for key in keys:
            doc, title = [], []
            for token in docs[key]:
                doc.append(token)
            for token in titles[key]:
                title.append(token.lemma)
            tokenized[key] = (title, doc)
        i += 1
    else:
        break

## Load in data.h5

In [90]:
#including parse takes 6 hours and doesn't do anything
config = {
    'mask': './**/**/*.posts',
    'train_mask': './**/**/TRAIN.txt',
    'test_mask': './**/**/TEST.txt',
    'dev_mask': './**/**/DEV.txt',
    'sample_mask': './SAMPLE.txt',
    #'parse':'./parses/*.parse',
    #'parsed':True
}

load = store.Store('data.h5', config=config, overwrite = False)
data = load.data

## Calculate Lengths

In [ ]:
#this can be replaced with read_parses() if it saves "tok.sents" and "tok.rank"

import spacy
nlp = spacy.load('en')

In [ ]:
#this can be replaced with read_parses() if it saves "tok.sents"
#currently takes ~3 hours to run

df = pandas.DataFrame(columns=('post_id', 'num_sent', 'avg_len', 'spelling', 'accuracy'))

for index, row in data.iterrows():
    #read with spacy, decode into unicode
    spacy_doc = nlp(row['text'].decode("utf-8"))
    
    #add tokens if their spacy rank isn't 0 (we could set a threshold here as well)
    no_typos = [tok for tok in spacy_doc if tok.rank!=0]    
    
    #simple counts (number of sentences, and avg lengh of sentence)
    num_sent = len(list(spacy_doc.sents))
    avg_len_sent =  sum([len(sent) for sent in list(spacy_doc.sents)])/len(list(spacy_doc.sents))  if len(list(spacy_doc.sents))> 0 else 0
    
    #most appear to have values over 90% (aka little typos).  Chose to set 0 length docs to 1.  
    accuracy = float(len(no_typos))/len(spacy_doc) if len(spacy_doc) > 0 else 1
    
    #append to new DF in the same row order
    df.loc[index] = row['post_id'], num_sent, avg_len_sent, no_typos, accuracy

print df
    

## TODO BoW BoDependencies, Subreddits

## Save 

In [ ]:
store.create_doc_feature(store.data ,df['post_id', 'num_sent', 'avg_len', 'spelling', 'accuracy'])

## Potpourri Analysis

In [56]:
#can we trust .rank?  I think yes.  High ranking words are pronouns, punctuation.  Low ranking words are proper nouns.  Misspellings are generally 0s (somethines high numbers)
words = nlp(u'suicide death die I ugh hate myself me wtf asfab beelzebub shakespeare javert misppeling reddit')

[tok.rank for tok in words]


[2594, 736, 683, 4, 8007, 405, 355, 49, 3075, 0, 264328, 80517, 436651, 0, 465]